In [29]:
from lightgbm import LGBMClassifier
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# List of parameter dictionaries to test
l = [
    {
        'n_estimators': 500,
        'learning_rate': 0.05,
        'max_depth': 10,
        'num_leaves': 50,
        'min_data_in_leaf': 30,
        'feature_fraction': 0.7,
        'bagging_fraction': 0.9,
        'bagging_freq': 5,
        'lambda_l1': 0.05,
        'lambda_l2': 0.05,
        'random_state': 42,
        'verbosity': -1
    }
]

# Load the dataset
df = pd.read_csv('train.csv')

# Select only the soil_type columns
soil_columns = [col for col in df.columns if col.startswith('Soil_Type')]

# Count the number of 0's and 1's in each soil_type column
zero_one_counts = pd.DataFrame({
    'Zeros': (df[soil_columns] == 0).sum(),
    'Ones': (df[soil_columns] == 1).sum()
})


# Check for NaNs and fill if needed
df = df.fillna(0)  # Replace NaNs with 0, or use another method as appropriate

# Step 1: Apply log transformation to skewed columns, handling negative values by adding a constant
skewed_columns = [
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Horizontal_Distance_To_Fire_Points',
    'Hillshade_3pm',
    "Hillshade_9am",
    "Hillshade_Noon"
]

sparse_columns = zero_one_counts[zero_one_counts['Ones'] < 10].index
df = df.drop(columns=sparse_columns)

# Apply the log transformation and handle negative values by shifting
for col in skewed_columns:
    # Shift values if necessary to make them positive before applying log
    min_value = df[col].min()
    shift = 1 - min_value if min_value <= 0 else 0
    df[f'log_{col}'] = np.log(df[col] + shift + 1)

# Step 2: Drop original skewed columns
df = df.drop(columns=skewed_columns)


# Separating the target column 'Cover_Type' from the features
X = df.drop(columns=['Cover_Type'])  # Drop target and non-feature columns
y = df['Cover_Type']  # Target column

# Normalize the numerical features
scaler = StandardScaler()
X_scaled = X.copy()

# List of numerical columns to be scaled
numerical_columns = [
    'Elevation', 'Aspect', 'Slope', 'log_Horizontal_Distance_To_Hydrology',
    'log_Vertical_Distance_To_Hydrology', 'log_Horizontal_Distance_To_Roadways',
    'log_Hillshade_9am', 'log_Hillshade_Noon', 'log_Hillshade_3pm', 
    'log_Horizontal_Distance_To_Fire_Points'
]

# Apply scaling only to the numerical columns
X_scaled[numerical_columns] = scaler.fit_transform(X[numerical_columns])

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


# Initialize the LGBMClassifier with the current parameters
lgbm_model = LGBMClassifier(**l[0])

# Train the model
lgbm_model.fit(X_train, y_train)

# Make predictions
y_pred = lgbm_model.predict(X_test)
y_pred_train = lgbm_model.predict(X_train)
# Evaluate the model's performance
test_accuracy = accuracy_score(y_test, y_pred)
train_accuracy = accuracy_score(y_train, y_pred_train)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")


Train Accuracy: 1.0000
Test Accuracy: 0.8962


In [30]:
# Initialize the LGBMClassifier with the current parameters
lgbm_model = LGBMClassifier(**l[0])

# Train the model
lgbm_model.fit(X_scaled, y)

LGBMClassifier(bagging_fraction=0.9, bagging_freq=5, feature_fraction=0.7,
               lambda_l1=0.05, lambda_l2=0.05, learning_rate=0.05, max_depth=10,
               min_data_in_leaf=30, n_estimators=500, num_leaves=50,
               random_state=42, verbosity=-1)

In [32]:
df_subs = pd.read_csv('test-full.csv')
df_subs = df_subs.drop(columns=sparse_columns)

# Apply the log transformation and handle negative values by shifting
for col in skewed_columns:
    # Shift values if necessary to make them positive before applying log
    min_value = df_subs[col].min()
    shift = 1 - min_value if min_value <= 0 else 0
    df_subs[f'log_{col}'] = np.log(df_subs[col] + shift + 1)

# Step 2: Drop original skewed columns
df_subs = df_subs.drop(columns=skewed_columns)
df_subs_scaled = df_subs.copy()
df_subs_scaled[numerical_columns] = scaler.transform(df_subs[numerical_columns])

In [33]:
submission_preds = pd.DataFrame(lgbm_model.predict(df_subs_scaled), columns=['Cover_Type'])

In [34]:
df.to_csv('filename.csv', index=False)

In [35]:
print(submission_preds.shape)
submission_preds.head()

(581012, 1)


,Cover_Type
0,5
1,5
2,2
3,2
4,5


In [36]:
submission_preds.to_csv('submission07-11-24.csv', index=True)

In [40]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('submission07-11-24.csv')

# Increment the 'Id' column by 1
df['Id'] = df['Id'] + 1

# Save the modified DataFrame back to CSV
df.to_csv('yourfile_modified.csv', index=False)
